## Installing Requirements

In [ ]:
! pip install openai
! pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 756.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210095 sha256=b28e98e752ff4dc7f119a7d7b803abdb544dfabb022944dfb7acab814441832c
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


## Creating Datasets

In [ ]:
input_file = "/kaggle/input/parsinlu-foods-dataset/food.jsonl"

with open(input_file) as f:
    lines = f.read().splitlines()

In [ ]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [ ]:
import json
df_inter['json_element'].apply(json.loads)

df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final.head(n=10)

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3,حبوبات و سویا,NaN,NaN,NaN,NaN,NaN,NaN
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1,شربت و آبمیوه,NaN,NaN,NaN,NaN,NaN,NaN
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2,تن ماهی,2,NaN,NaN,NaN,NaN,NaN
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2,گوشت مرغ,-2,-2,NaN,NaN,NaN,NaN
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1,نوشابه,NaN,NaN,-1,1,NaN,NaN
5,طعم و مزه عسل رو داره من توی شگفت انگیز خریدم ...,3,عسل,1,NaN,NaN,NaN,-1,NaN
6,پسته بو داده و نمکین گلستان رو تازه استفاده کر...,1,تخمه و مغز طعم‌دار,NaN,1,NaN,NaN,NaN,NaN
7,بسیار خوش طعم و عالیه یه تن ماهی باکیفیت و خوشمزه,2,تن ماهی,2,2,NaN,NaN,NaN,NaN
8,کیفیت خوبی داره اما قیمتش بالاست. خرید میگو تا...,3,ماهی، میگو و خاویار,NaN,1,NaN,-1,NaN,NaN
9,"تاریخ روز گذشته بود, خیلی کوچک بود و تازه نبود...",-2,گوشت مرغ,NaN,-2,NaN,NaN,NaN,NaN


In [ ]:
df_final['sentiment'].value_counts()

-2    462
2     440
1     420
-1    381
3     184
0      30
Name: sentiment, dtype: int64

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              1917 non-null   object
 1   sentiment           1917 non-null   object
 2   category            1917 non-null   object
 3   aspects.طعم         886 non-null    object
 4   aspects.کیفیت       553 non-null    object
 5   aspects.ارزش غذایی  89 non-null     object
 6   aspects.ارزش خرید   414 non-null    object
 7   aspects.بسته بندی   156 non-null    object
 8   aspects.ارسال       64 non-null     object
dtypes: object(9)
memory usage: 134.9+ KB


In [ ]:
df_final.describe()

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
count,1917,1917,1917,886,553,89,414,156,64
unique,1917,6,20,6,6,6,6,5,4
top,من یه مدته فقط از این محصول استفاده میکنم اما ...,-2,تن ماهی,1,1,1,1,1,-1
freq,1,462,106,269,179,39,176,62,24


In [ ]:
df_final = df_final[['review', 'sentiment']]
df_final.head(n=5)

,review,sentiment
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1


In [ ]:
random_state = 18

samples_count = 1
sample_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=samples_count, random_state=random_state))
sample_prompts

,,review,sentiment
sentiment,,,
-1,17,پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در...,-1
-2,836,از این برند انتظار بیشتری داشتم خیلی خشک و بی ...,-2
0,1015,تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم...,0
1,238,دفعه های قبل که سفارش دادم بو میداد ولی ایندفع...,1
2,499,به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی...,2
3,916,از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه...,3


## OpenAI Prompt Methodology

In [ ]:
import openai
openai.api_key = "..."

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## Prompts

In [ ]:
import time

max_retries = 5

In [ ]:
label_dict = {
    "-2" : "NEGATIVE",
    "-1" : "NEGATIVE",
    "0"  : "NEUTRAL",
    "1"  : "POSITIVE",
    "2"  : "POSITIVE",
    "3"  : "OTHER",
}

In [ ]:
per_to_eng_dict = {
    "مثبت" : "POSITIVE",
    "خنثی" : "NEUTRAL",
    "منفی" : "NEGATIVE",
    "سایر" : "OTHER",
}

In [ ]:
eng_to_per_dict = {
    "POSITIVE"  : "مثبت",
    "NEUTRAL"   : "خنثی",
    "NEGATIVE"  : "منفی",
    "OTHER"     : "سایر",
}

In [ ]:
acceptable_classes = [
    "NEGATIVE",
    "NEUTRAL",
    "POSITIVE",
    "OTHER",
]

In [ ]:
prompt_01 = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \

Review: SAMPLE_REVIEW
Sentiment:
"""

In [ ]:
prompt_02 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Review: SAMPLE_REVIEW
Sentiment:
"""

In [ ]:
prompt_03 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

نظر: SAMPLE_REVIEW
احساس:
"""

In [ ]:
prompt_04 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \

نظر: SAMPLE_REVIEW
احساس:
"""

## Zero-shot Sample Test

### Prompt #01

### Prompt #02

### Prompt #03

### Prompt #04

## Zero-shot Metric

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter
import numpy as np

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=0)
    precision = precision_score(actual, predicted, average='weighted', zero_division=0)
    recall = recall_score(actual, predicted, average='weighted', zero_division=0)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = list(set(actual + predicted))
    classes.append('All Classes')
    metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

    probability_table = pd.DataFrame(index=metrics, columns=classes)

    for class_label in classes:
        if class_label == 'All Classes':
            probability_table[class_label]["F1 Score"] = f1
            probability_table[class_label]["Precision"] = precision
            probability_table[class_label]["Recall"] = recall
        else:
            class_precision = precision_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_recall = recall_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_f1 = f1_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            probability_table[class_label]["F1 Score"] = class_f1
            probability_table[class_label]["Precision"] = class_precision
            probability_table[class_label]["Recall"] = class_recall
    probability_table.fillna(value="-", inplace = True)
    probability_table['All Classes']["Accuracy"] = accuracy
    return probability_table

In [ ]:
test_count = 190
test_df = df_final.sample(n=test_count, random_state=random_state)
test_df

,review,sentiment
1634,فقط بدیه این محصول اینه که حد اکثر ۱۰ عدد میشه...,3
1891,نسبت به قیمت جدیدش قهوه های بهتری رو توی این ر...,-1
858,قطر 1 این برندو جایی ندیدم.اتفاقا دنبالش بودم ...,1
684,رگ گیری که اصلا نشده بود و کار خیلی سختی بود ر...,-2
446,طعم خیلی خوب بود و قیمتش درشگفت انگیز عالی بود,2
...,...,...
577,خیلی رقیق است ولی طعمش خوبه. مقدارش هم روی کال...,3
172,اصلا ارزش امتحان کردن هم نداره چه برسه به خرید...,-2
160,از بس که شیرینی مصنوعی بهش زدن که دل رو می زنه.,-1
690,این حلوا ارده نیست و حلوا شکری است و اگه میدون...,-1


In [ ]:
test_df.to_csv('test_df.csv',index=True)

### Prompt #01

In [ ]:
# from alive_progress import alive_bar

# zero_01_actual = []
# zero_01_predicted = []
# zero_01_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_01.replace("SAMPLE_REVIEW", row['review']))
#                 zero_01_responses.append(response)
#                 zero_01_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         zero_01_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     zero_01_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# zero_01_results = calculate_metrics(zero_01_actual, zero_01_predicted)
# zero_01_results

In [ ]:
# np.save('zero_01_actual', zero_01_actual)

In [ ]:
# np.save('zero_01_predicted', zero_01_predicted)

In [ ]:
# np.save('zero_01_responses', zero_01_responses)

In [ ]:
# zero_01_results.to_csv('zero_01_results.csv',index=True)

### Prompt #02

In [ ]:
# from alive_progress import alive_bar

# zero_02_actual = []
# zero_02_predicted = []
# zero_02_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_02.replace("SAMPLE_REVIEW", row['review']))
#                 zero_02_responses.append(response)
#                 zero_02_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         zero_02_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     zero_02_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# zero_02_results = calculate_metrics(zero_02_actual, zero_02_predicted)
# zero_02_results

In [ ]:
# np.save('zero_02_actual', zero_02_actual)

In [ ]:
# np.save('zero_02_predicted', zero_02_predicted)

In [ ]:
# np.save('zero_02_responses', zero_02_responses)

In [ ]:
# zero_02_results.to_csv('zero_02_results.csv',index=True)

### Prompt #03

In [ ]:
# from alive_progress import alive_bar

# zero_03_actual = []
# zero_03_predicted = []
# zero_03_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_03.replace("SAMPLE_REVIEW", row['review']))
#                 zero_03_responses.append(response)
#                 zero_03_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         zero_03_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     zero_03_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# zero_03_results = calculate_metrics(zero_03_actual, zero_03_predicted)
# zero_03_results

In [ ]:
# np.save('zero_03_actual', zero_03_actual)

In [ ]:
# np.save('zero_03_predicted', zero_03_predicted)

In [ ]:
# np.save('zero_03_responses', zero_03_responses)

In [ ]:
# zero_03_results.to_csv('zero_03_results.csv',index=True)

### Prompt #04

In [ ]:
# from alive_progress import alive_bar

# zero_04_actual = []
# zero_04_predicted = []
# zero_04_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_04.replace("SAMPLE_REVIEW", row['review']))
#                 zero_04_responses.append(response)

#                 for key in per_to_eng_dict.keys():
#                     if key in response:
#                         response = per_to_eng_dict[key]
#                         break

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         zero_04_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     zero_04_predicted.append('OTHER')

#                 zero_04_actual.append(label_dict[row['sentiment']])

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# zero_04_results = calculate_metrics(zero_04_actual, zero_04_predicted)
# zero_04_results

In [ ]:
# np.save('zero_04_actual', zero_04_actual)

In [ ]:
# np.save('zero_04_predicted', zero_04_predicted)

In [ ]:
# np.save('zero_04_responses', zero_04_responses)

In [ ]:
# zero_04_results.to_csv('zero_04_results.csv',index=True)

## Examples

In [ ]:
def create_examples_df(sample_size=1):
    example_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=sample_size, random_state=random_state))
    example_prompts = example_prompts.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return example_prompts

In [ ]:
def create_examples_prompt():
    examples_df = create_examples_df()
    result = ""
    result_persian = ""
    results_semi_persian = ""
    i = 1
    for index, row in examples_df.iterrows():
        result += f"Example {i}:\n"
        result += f"Review: {row['review']}\n"
        result += f"Sentiment: {label_dict[row['sentiment']]}\n"

        results_semi_persian += f"مثال {i}:\n"
        results_semi_persian += f"نظر: {row['review']}\n"
        results_semi_persian += f"احساس: {label_dict[row['sentiment']]}\n"

        result_persian += f"مثال {i}:\n"
        result_persian += f"نظر: {row['review']}\n"
        result_persian += f"احساس: {eng_to_per_dict[label_dict[row['sentiment']]]}\n"

        i += 1

    return result, results_semi_persian, result_persian

In [ ]:
examples, results_semi_persian, examples_persian = create_examples_prompt()
print(examples)
print("============================")
print()
print(examples_persian)
print("============================")
print()
print(results_semi_persian)

Example 1:
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد ویژه خوبه
Sentiment: OTHER
Example 2:
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Sentiment: POSITIVE
Example 3:
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Sentiment: NEGATIVE
Example 4:
Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Sentiment: NEGATIVE
Example 5:
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Sentiment: POSITIVE
Example 6:
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Sentiment: NEUTRAL


مثال 1:
نظر: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد ویژه خوبه
احساس: سایر
مثال 2:
نظر: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
احساس: مثبت
مثال 3:
نظر: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
احساس: منفی
مثال 4:
نظر: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
احساس: منفی
مثال 5:
نظر: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود

## Few-Shot Prompts

### Prompt #01

In [ ]:
prompt_01_few = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \

Examples: \
{examples}

Review: SAMPLE_REVIEW
Sentiment:
"""

### Prompt #02

In [ ]:
prompt_02_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Examples: \
{examples}

Review: SAMPLE_REVIEW
Sentiment:
"""

### Prompt #03

In [ ]:
prompt_03_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Examples: \
{results_semi_persian}

نظر: SAMPLE_REVIEW
احساس:
"""

### Prompt #04

In [ ]:
prompt_04_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \

Examples: \
{examples_persian}

نظر: SAMPLE_REVIEW
احساس:
"""

## Few-shot Sample Test

## Few-shot Metric

### Prompt #01

In [ ]:
# from alive_progress import alive_bar

# few_01_actual = []
# few_01_predicted = []
# few_01_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_01_few.replace("SAMPLE_REVIEW", row['review']))
#                 few_01_responses.append(response)
#                 few_01_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         few_01_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     few_01_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30*retries)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# few_01_results = calculate_metrics(few_01_actual, few_01_predicted)
# few_01_results

In [ ]:
# np.save('few_01_actual', few_01_actual)

In [ ]:
# np.save('few_01_predicted', few_01_predicted)

In [ ]:
# np.save('few_01_responses', few_01_responses)

In [ ]:
# few_01_results.to_csv('few_01_results.csv',index=True)

### Prompt #02

In [ ]:
# from alive_progress import alive_bar

# few_02_actual = []
# few_02_predicted = []
# few_02_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_02_few.replace("SAMPLE_REVIEW", row['review']))
#                 few_02_responses.append(response)
#                 few_02_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         few_02_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     few_02_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30*retries)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# few_02_results = calculate_metrics(few_02_actual, few_02_predicted)
# few_02_results

In [ ]:
# np.save('few_02_actual', few_02_actual)

In [ ]:
# np.save('few_02_predicted', few_02_predicted)

In [ ]:
# np.save('few_02_responses', few_02_responses)

In [ ]:
# few_02_results.to_csv('few_02_results.csv',index=True)

### Prompt #03

In [ ]:
# from alive_progress import alive_bar

# few_03_actual = []
# few_03_predicted = []
# few_03_responses = []

# with alive_bar(test_count, force_tty=True) as bar:
#     for index, row in test_df.iterrows():
#         retries = 0
#         while retries < max_retries:
#             try:
#                 response = get_completion(prompt_03_few.replace("SAMPLE_REVIEW", row['review']))
#                 few_03_responses.append(response)
#                 few_03_actual.append(label_dict[row['sentiment']])

#                 flag = False
#                 response = response.upper()
#                 for label in acceptable_classes:
#                     if label in response:
#                         few_03_predicted.append(label)
#                         flag = True
#                         break

#                 if not flag:
#                     few_03_predicted.append('OTHER')

#                 time.sleep(20)
#                 break

#             except Exception as e:
#                 print(f"An error occurred: {str(e)}")
#                 retries += 1
#                 print(f"Retrying ({retries}/{max_retries})...")
#                 time.sleep(30*retries)

#         if retries == max_retries:
#             print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

#         bar()

In [ ]:
# few_03_results = calculate_metrics(few_03_actual, few_03_predicted)
# few_03_results

In [ ]:
# np.save('few_03_actual', few_03_actual)

In [ ]:
# np.save('few_03_predicted', few_03_predicted)

In [ ]:
# np.save('few_03_responses', few_03_responses)

In [ ]:
# few_03_results.to_csv('few_03_results.csv',index=True)

### Prompt #04

In [ ]:
from alive_progress import alive_bar

few_04_actual = []
few_04_predicted = []
few_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_04_few.replace("SAMPLE_REVIEW", row['review']))
                few_04_responses.append(response)

                for key in per_to_eng_dict.keys():
                    if key in response:
                        response = per_to_eng_dict[key]
                        break

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    few_04_predicted.append('OTHER')

                few_04_actual.append(label_dict[row['sentiment']])

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30*retries)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 190/190 [100%] in 1:05:26.3 (0.05/s) 


In [ ]:
few_04_results = calculate_metrics(few_04_actual, few_04_predicted)
few_04_results

,POSITIVE,NEGATIVE,NEUTRAL,OTHER,All Classes
Accuracy,-,-,-,-,0.7
F1 Score,[0.8342857142857143],[0.8251748251748252],[0.05555555555555556],[0.0],0.712857
Precision,[0.9358974358974359],[0.7662337662337663],[0.02857142857142857],[0.0],0.744116
Recall,[0.7525773195876289],[0.8939393939393939],[1.0],[0.0],0.7


In [ ]:
np.save('few_04_actual', few_04_actual)

In [ ]:
np.save('few_04_predicted', few_04_predicted)

In [ ]:
np.save('few_04_responses', few_04_responses)

In [ ]:
few_04_results.to_csv('few_04_results.csv',index=True)